In [ ]:
#
# Attempt at a modified version D onto a GPU.
#
# This won't work because all the threads in a warp must execute the exact same 
# instructions which may not happen here because of branching and jump behavior
#
import os
# needs to appear before CUDA import
os.environ["NUMBA_ENABLE_CUDASIM"] = "0"
# set to "1" for more debugging, but slower performance
os.environ["NUMBA_CUDA_DEBUGINFO"] = "0"

from numba import jit, njit, cuda , prange
import numba
import numpy as np
import sys

# original problem
# registers = [729, 0, 0]
# program = [0,1,5,4,3,0]

# enabling the JIT may cause print statements not to print lists as expected
# Return array passed in as a parameter because GPU can only modify passed-in arrays can't return anything
# This kernel function is invoked once per thread
@cuda.jit()
def loop_em(program, num_match_elements, return_array, start, end):
    
    register_a = 0
    register_b = 0
    register_c = 0

    # cuda.threadIdx.x should range [0..cuda.blockDim.x] [0..256]
    # cuda.blockIdx.x should range  [0..cuda.gridDim.x] [0..(4*38)]
    # cuda unique thread identifier
    global_thread_id = cuda.grid(1)
    # print(cuda.blockDim.x * 10000000000 + cuda.blockIdx.x * 1000000 + cuda.threadIdx.x )

    # handle if not exactly divisible
    # how many numbers we sweep in each thread
    cycles_per_thread = ((end - start) // cuda.gridsize(1))+1

    # This has to be a constant due to CUDA rules so we pick something longer than our test program
    # Create an array, and assume it is no longer than our expected program.  This is a testing shortcut
    output = cuda.local.array(20, np.int64) # output array

    for cycle_index in range(cycles_per_thread):
        # which thread block are we in?
        global_cycle_id = cycle_index + (global_thread_id * cycles_per_thread)
        # handle the fact the batch size is not an exact factor of the total range
        if (global_cycle_id <= end):
            # This program is 8 octal operands so the A register needs to be 8 octal digits.
            # Registers are reset at the start of every run
            register_a = global_cycle_id
            register_b = 0
            register_c = 0
            # This is a hack to create an array but it should be calculated from num_match_elements
            # Use a persistent index because we can't append to the list
            output_index = 0 
            for i in range(num_match_elements):
                output[i] = -1
            # Program address
            address_ptr = 0

            while (address_ptr < len(program)):
                # numba says these are int64
                operator = program[address_ptr]
                operand = program[address_ptr+1]
                next_address_ptr = address_ptr+2
                match (operator):
                    case 0: # adv division register_a ~/ 2^comboOperand
                        register_a = register_a // 2 ** resolve_operand(register_a, register_b, register_c, operand)
                    case 1: # bxl bitwise XOR (registerB , operand)
                        register_b = register_b ^ operand
                    case 2: # bst operand modulo 8
                        register_b = resolve_operand(register_a, register_b, register_c, operand) % 8
                    case 3: # jnz jump not zero
                        if (register_a != 0):
                            next_address_ptr =  operand
                    case 4: #bxc bitwise xor reg b, reg c
                        register_b = register_b ^ register_c
                    case 5: # out % modulo 8
                        value = resolve_operand(register_a, register_b, register_c, operand) %8
                        output[output_index]= value
                        output_index += 1
                    case 6: # BDV integer division on A, stored in B
                        divisor = 2 ** resolve_operand(register_a, register_b, register_c, operand)
                        register_b = register_a // divisor
                    case 7: # CDV
                        divisor = 2 ** resolve_operand(register_a, register_b, register_c, operand)
                        register_c = register_a // divisor
                    case _:
                        #print('oh no')
                        result = -1   
                address_ptr = next_address_ptr
                # This exists so I can experiment with different lengths while experimenting
                if (match_sub_list(program,num_match_elements,output)):
                    return_array[global_thread_id]= global_cycle_id
                    return
            # return_array[global_thread_id]= output_index
            # return
    # no matches if we get here
    return_array[global_thread_id]=-1
    return

@cuda.jit(device=True)
# pass everything in so we don't get a capture error
# implements the value/register value rules for an operand value
def resolve_operand(register_a, register_b, register_c,operand):
    if (operand < 4):
        return operand
    elif (operand ==4):
        return register_a
    elif (operand ==5):
        return register_b
    elif (operand ==6):
        return register_c
    else:
        return 0

@cuda.jit(device=True)
# match some subset of the values in two lists
def match_sub_list(program, num_match_elements, output_array) -> bool:
    for i in range(len(program)):
        if (i >= num_match_elements):
            return True
        if (program[i] != output_array[i]):
            return False
    return True




In [ ]:
%%time
# num_blocks_per_grid number of blocks in a grid (SMUs in GPUs or cores in CPUs)
# num_threads_per_block = number of threads per core or number of threads that can run in an SMU
#
# thread id is the id within a block
# block id is the id within a grid
# block width is the number of threads per block
# blocks per grid == SMUs * some occupancy factor
# threads per block is some multiple of cores per SMU - they share memory

# Titan RTX has 72 SMs with 64 processors -> 4608 cores
num_blocks_per_grid = (72*1)
num_threads_per_block = (64*2)
# RTX 3060 TI has 38 SMs with 128 processors each -> 4864 cores
# Used an occupancy calculator https://xmartlabs.github.io/cuda-calculator/ 
# It gave us 38*8
num_blocks_per_grid = (38*4)
num_threads_per_block = (1)

program = np.array([2,4,1,3,7,5,4,7,0,3,1,5,5,5,3,0])
total_num_threads = (num_blocks_per_grid * num_threads_per_block)
return_array = np.zeros(total_num_threads, dtype=int)
print(f"Created return array sized {total_num_threads} for (b/g:{num_blocks_per_grid} x t/b:{num_threads_per_block})")
# print the entire results array in octal - useful while debugging
np.set_printoptions(formatter={'int':oct},threshold=sys.maxsize)

# copy the program and the results array to the GPU
g_program = cuda.to_device(program)
g_return_array = cuda.to_device(return_array)
loop_em[num_blocks_per_grid,num_threads_per_block](        
        g_program,
        7,
        g_return_array,
        int(0o1000000000000000),
        int(0o7777777777777777),
       )
return_array = g_return_array.copy_to_host()
print(return_array)

# 16 digits octal
# 16th digit must be 1 otherwise the return is shorter than the program

# 3060 ti
# 05:49 with (38*8 x 128*2) matching 6 digits
# 01:05 with (38*8 x 1    ) matching 6 digits
# 03:09 with (38*4 x 128*2) matching 6 digits 
# 00:59 with (38*4 x 1    ) matching 6 digits
# 01:42 with (38*2 x 128*2) matching 6 digits 
# 00:59 with (38*2 x 1    ) matching 6 digits
# 01:18 with (38*1 x 128*2) matching 6 digits
# 01:10 with (38*1 x 128*1) matching 6 digits alt run
# 01:12 with (38*1 x 128*1) matching 6 digits
# 00:59 with (38*1 x 1    ) matching 6 digits
# 00:31 with ( 1   x 128*2) matching 6
# 00:29 with ( 1   x 128*1) matching 6
# 00:12 with ( 1   x 1    ) matching 6
#
# 17:36 with (38*4 x 1    ) matching 7 digits
# 21:06 with (38*1 x 128*1) matching 7 digits
# 17:30 with (38*1 x 1    ) matching 7 digits
# 13:51 with ( 1   x 128*1) matching 7 digits
# 14:08 with ( 1   x 32   ) matching 7 digits
# 06:39 with ( 1   x 1    ) matching 7 digits
#
# 67:39 with ( 1   x 1    ) matching 8 digits
#
#